In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.impute import SimpleImputer
import os

# Set working directory
os.chdir('E:\\JESIN\\DOCUMENTS\\scma\\A2a')
print(os.getcwd())

# Load the dataset
data = pd.read_csv("NSSO68.csv")

# Subset data to state assigned
subset_data = data[data['state_1'] == 'UP'][['foodtotal_v', 'hhdsz', 'Regular_salary_earner', 'MPCE_MRP', 'MPCE_URP', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data)


E:\JESIN\DOCUMENTS\scma\A2a


C:\Users\user\AppData\Local\Temp\ipykernel_19940\3368925511.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("NSSO68.csv")


       foodtotal_v  hhdsz  Regular_salary_earner  MPCE_MRP  MPCE_URP  \
29288   777.287833      6                    2.0   2418.53   2042.00   
29289   504.344250      4                    2.0   1968.38   1503.50   
29290   548.630000     10                    2.0   1170.03   1023.30   
29291   715.650500      2                    2.0   1702.70   1656.50   
29292   434.551583     12                    1.0   1258.96   1167.50   
...            ...    ...                    ...       ...       ...   
94159   352.571333      6                    2.0    880.05    951.83   
94160   369.088600      5                    2.0   3823.12  34518.80   
94161   290.485125      8                    2.0    545.02    705.00   
94162   285.053000      8                    2.0    633.28    532.75   
94163   362.775833      6                    2.0    963.97   1351.83   

       Possess_ration_card  Education  No_of_Meals_per_day  
29288                  1.0       12.0                  2.0  
29289        

In [5]:
# Check for missing values
print(subset_data['hhdsz'].isna().sum())
print(subset_data['Regular_salary_earner'].isna().sum())
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(subset_data['Education'].isna().sum())
print(subset_data['No_of_Meals_per_day'].isna().sum())

# Impute missing values with mean
imputer = SimpleImputer(strategy='mean')
subset_data['Possess_ration_card'] = imputer.fit_transform(subset_data[['Possess_ration_card']])

print("Possess_ration_card:")
print(subset_data['Possess_ration_card'].isna().sum())

0
0
0
0
0
0
0
Possess_ration_card:
0


In [6]:
# Fit the regression model
X = subset_data[['hhdsz', 'Regular_salary_earner', 'MPCE_MRP', 'MPCE_URP', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = subset_data['foodtotal_v']

model = sm.OLS(y, X).fit()

# Print the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            foodtotal_v   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     1302.
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               0.00
Time:                        13:17:15   Log-Likelihood:                -61381.
No. Observations:                9015   AIC:                         1.228e+05
Df Residuals:                    9007   BIC:                         1.228e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   361.01

In [7]:
# Check for multicollinearity using Variance Inflation Factor (VIF)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)  # VIF Value more than 8 is problematic

                 feature         VIF
0                  const  105.477846
1                  hhdsz    1.098855
2  Regular_salary_earner    1.138218
3               MPCE_MRP    2.068354
4               MPCE_URP    1.968635
5    Possess_ration_card    1.048881
6              Education    1.230296
7    No_of_Meals_per_day    1.004672


In [8]:
# Extract the coefficients from the model
coefficients = model.params

# Construct the equation
equation = f"y = {coefficients[0]:.2f}"
for i in range(1, len(coefficients)):
    equation += f" + {coefficients[i]:.6f}*x{i}"

# Print the equation
print(equation)

y = 361.02 + -12.963010*x1 + -14.608830*x2 + 0.072781*x3 + 0.059190*x4 + -48.084503*x5 + 7.634267*x6 + 49.872590*x7


C:\Users\user\AppData\Local\Temp\ipykernel_19940\337899916.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  equation = f"y = {coefficients[0]:.2f}"
C:\Users\user\AppData\Local\Temp\ipykernel_19940\337899916.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  equation += f" + {coefficients[i]:.6f}*x{i}"
